In [1]:
import numpy as np
from sparse_vector import *
from vector_polynomial_isomorphism import *

ModuleNotFoundError: No module named 'sparse_vector'

We note that we are using the base field `QQ` for all computations currently

In [2]:
# Let us initialize constants and vector spaces.

N = 9
n = 3
d = 3
indeterminates = [var("y_%d%d" % ((i+1),(j+1))) for i in range(n) for j in range(n)]
R = PolynomialRing(QQ, indeterminates)
R.inject_variables()

dimV = binomial(N + d - 1, d)
V = VectorSpace(QQ, dimV)
glN = MatrixSpace(QQ, N)

vp_iso = vector_polynomial_isomorphism(R, V, indeterminates, d)

Defining y_11, y_12, y_13, y_21, y_22, y_23, y_31, y_32, y_33


NameError: name 'vector_polynomial_isomorphism' is not defined

Note $\hat{\sigma_{r}}(\mathbb{P}^{n-1} \times \mathbb{P}^{n-1}) = \mathbb{V}( (r+1)\times(r+1) \text{ minors})$

In [3]:
# Let us inititalize the list of polynomials, f
# In our case, r=3.

M = MatrixSpace(R, n)
A = M(np.array(indeterminates).reshape(-1,n))
if n <= 3:
    f = [R(A.determinant())]
else:
    f = A.minors(4)
    
grad_f = [fi.gradient() for fi in f]

In [4]:
# action of basis matrix E_ab in glN onto homogeneous polynomial f[i]
def E_action(a, b, i):
    sum = -1*indeterminates[b]*grad_f[i][a]
    return R(sum)

In [5]:
# W = span{f}

f_vec = [vp_iso.polynomial_to_vector(fi) for fi in f]
W = V.subspace(f_vec)

print(W)
p = W.dimension()

Vector space of degree 3876 and dimension 1 over Rational Field
Basis matrix:
1 x 3876 dense matrix over Rational Field


In [6]:

W_perp = W.complement()

print(W_perp)

g = W_perp.basis()
q = W_perp.dimension()

Vector space of degree 3876 and dimension 3875 over Rational Field
Basis matrix:
3875 x 3876 dense matrix over Rational Field


We note `M_f` $:= \tilde{M}_f$

In [7]:
# Pre-processing memoization of entries of M_f

# E_action_f[l][i][j] = (E_ij)(f_l)
E_action_f = [[[sparsify(vp_iso.polynomial_to_vector(E_action(i,j,l))) for i in range(N)] for j in range(N)] for l in range(p)]

# convert g to sparse representation
g_sparse = [sparsify(g[i]) for i in range(q)]

In [8]:

# Construct M_f
M_f = zero_matrix(QQ, p*q, N^2)
for l in range(p):
    for k in range(q):
        for i in range(N):
            for j in range(N):
                M_f[(p*l) + k,(N*i) + j] = sparse_inner_product(E_action_f[l][i][j], g_sparse[k])

print('rank(M_f) =', M_f.rank())
print('dim(ker(M_f)) =', glN.dimension() - M_f.rank())
print('2*(n^2)-1 =', 2*(N)-1)

rank(M_f) = 225
dim(ker(M_f)) = 31
2*(n^2)-1 = 31


In [12]:
## TESTING

kern = M_f.right_kernel().basis()

def X_action(X, f):
    grad_test = f.gradient()
    sum = 0
    for a in range(N):
        for b in range(N):
            sum += -X[N*a + b]*indeterminates[b]*grad_test[a]
    return R(sum)

for i in range(1):
    for j in range(q):
        X = kern[0]
        test = vp_iso.polynomial_to_vector(X_action(X,f[i])).inner_product(g[j])
        if test != 0:
            print(i, j)
            print(test)
            print(X_action(X,f[i]))
            print(vp_iso.vector_to_polynomial(g[j]))
            print(sparsify(vp_iso.polynomial_to_vector(X_action(X,f[i]))))
            print(sparsify(g[j]))
            print(sparse_inner_product(sparsify(g[j]),sparsify(vp_iso.polynomial_to_vector(X_action(X,f[i])))))